In [218]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2


In [219]:
DB_NAME = "redNotices_db"
DB_USER = "postgres"
DB_PASSWORD = "root"
DB_HOST = "localhost"
DB_PORT = "5432"

In [220]:
con = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port = DB_PORT
)
con.autocommit = True
cursor = con.cursor()

In [221]:
query_create_table = """ CREATE TABLE IF NOT EXISTS wanted_persons(
    entity_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    forename TEXT NOT NULL,
    date_of_birth TEXT NOT NULL,
    sex_id TEXT NOT NULL,
    place_of_birth TEXT,
    weight TEXT
    );
    """
cursor.execute(query_create_table)

In [222]:
def add_db_person(entity_id, name, forename, date_of_birth, sex_id, place_of_birth, weight):
    query_INSERT = f"INSERT INTO suclular (entity_id, name, forename, date_of_birth, sex_id, place_of_birth, weight) SELECT '{entity_id}', '{name}', '{forename}', '{date_of_birth}', '{sex_id}', '{place_of_birth}', '{weight}' WHERE NOT EXISTS ( SELECT * FROM wanted_persons WHERE entity_id='{entity_id}' and name='{name}' and forename='{forename}' and date_of_birth='{date_of_birth}' and sex_id='{sex_id}' and place_of_birth='{place_of_birth}' and weight='{weight}');"
    cursor.execute(query_INSERT)

In [223]:
headers = {
    'authority': 'ws-public.interpol.int',
    'accept': '*/*',
    'accept-language': 'tr,en-US;q=0.9,en;q=0.8,vi;q=0.7,fr;q=0.6',
    'origin': 'https://www.interpol.int',
    'referer': 'https://www.interpol.int/',
    'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
}

In [ ]:


for page_numb in range(9):
    response = requests.get(f'https://ws-public.interpol.int/notices/v1/red?resultPerPage=20&page={page_numb}', headers=headers)

    datas = response.json()

    # json dosyasındaki _embedded ve notices'in içerisine girdim.
    ad = datas["_embedded"]["notices"]

    # her sayfada 20 suçlu alınıyor 
    for i in range(20):
        #her suçlu loop'unda entity_id alınıp suçlu sayfasının url'i oluşturuluyor
        entity_id = ad[i]['entity_id'].replace("/", "-")

        # suçlu sayfası için istek gönderiliyor
        url = requests.get("https://ws-public.interpol.int/notices/v1/red/" + entity_id)

        # json formatına döndürülüyor
        persons = url.json()
        #kişilerin bilgileri alınıyor

        wanted_persons = {
            "name" : persons['name'],
            "forename": persons['forename'],
            "weight" : persons['weight'],
            "date_of_birth" : persons['date_of_birth'],
            "entity_id" : persons['entity_id'],
            "languages_spoken_ids" : persons['languages_spoken_ids'],
            "nationalities" : persons['nationalities'],
            "sex_id" : persons['sex_id'],
            "country_of_birth_id" : persons['country_of_birth_id'],
            "distinguishing_marks" : persons['distinguishing_marks'],
            "eyes_colors_id" : persons['eyes_colors_id'],
            "hairs_id" : persons['hairs_id'],
            "place_of_birth" : persons['place_of_birth']
             }
        

        print(wanted_persons["name"], wanted_persons["forename"], wanted_persons["date_of_birth"], wanted_persons["entity_id"], wanted_persons["sex_id"])

        add_db_person(wanted_persons["entity_id"],wanted_persons["name"], wanted_persons["forename"], wanted_persons["date_of_birth"], wanted_persons["sex_id"], wanted_persons["place_of_birth"], wanted_persons["weight"])
